#### Data Collection



start with scraping data to build csv file (beutiful soup)
https://www.pgatour.com/stats/stat/jcr:content/mainParsys/details.101.y2021.eon.t013.scontent.html

- in the network look for .details calls with an html file

  - score (round score)
  - player average score
  - greens in regulation
  - fairways in regulation
  - putts per round
  - up and down conversions
  - course design (stanley tompson)
  - player home course design
  - weather conditions
  - is a major event
  - player position in the golf rankings (may lead to more or less pressure)
  - tournment leading score, compare to round score to see if there is a diference

- I need to seperate categorical and numerical data and visual our data (cleaning data)
- Collect significant values to run through ML or linear regression models

NOTES
- try to be to vague and deal with too many averages (years is too big, pick a specific tournment)
IMPORTANT NOTES

- each stat is a 4 day average
- only players who make it to the final round (4 rounds) are considered



In [2]:
#IMPORTS
import pandas as pd
import requests
import time


### DATA COLLECTION

Let's begin by collecting data from 



Found the raw api collection of data in network
Checked the /robots.txt and saw that it was no allowed to scrap from there

Allowed to scrap from the stats directly

https://www.owgr.com/ -> no robot.txt disallowance


Plan for collecting data

- Want to predict total earning
- 1) get the season winning table
- 2) determine the tournment payouts (exclude majors?)
- 3) relationship to scoring average
  - 4) stokes gained on off the tee, on the fairway, around the green, on the green
  *Off-the-tee + approach-the-green + around-the-green + putting = strokes gained: total
- Are stroked gained more important on certain courses (links, stadium, parkland course)

  

- what I want, given a tournment (golf course type + winnings), weather (effect strokes gained), average stokes gained (all aspects)

Let's define a helper function to build our csv files based on our stat code and title. It will be important to rate limit our requests to not overwhelm the servers.

In [3]:
def data_frame_builder(title: str, statCd: int) -> pd.DataFrame:
    allDfs = []
    
    tournamentMap =  {
    "The Masters": '014',
    "U.S. Open" : '026',
    "The Memorial Tournament": '023',
    "RBC Canadian Open": '032',
    "AT&T Bryon Nelson": '019',
    "Travelers Championship": '034',
    "Wells Fargo Championship": '480'
    }
    
    for event, eventCd in tournamentMap.items():
        #rate limit our requests
        time.sleep(2)
        
        #build the url and create a get request for the html
        url = f'https://www.pgatour.com/stats/stat.{statCd}.y2022.eon.t{eventCd}.html'
        rs = requests.get(url)
        
        #use panadas to read <table> tags and generate a data frame (second table on the page is desired)
        scrapedDfs = pd.read_html(rs.text)
        currDf = scrapedDfs[1]
        
        #add event column for multi indexing
        currDf['EVENT'] = event
        
        #append df to array
        allDfs.append(currDf)
    
    completedDf = pd.concat(allDfs)
    
    #clean the NaN data
    cleanedDf = completedDf.dropna(axis=1, how='all')
    cleanedDf = cleanedDf.drop(cleanedDf.columns[0], axis = 1)
    
    return cleanedDf

Collect the following data frames from 2010 - 2020, please refer each table definitions belows.

Offical Money: Total compensation
Scoring Average:

Driving Distance:
Driving Accuracy: The percentage of time a tee shot comes to rest in the fairway (regardless of club).

Greens in Regulation: The percent of time a player was able to hit the green in regulation (greens hit in regulation/holes played). Note: A green is considered hit in regulation if any portion of the ball is touching the putting surface after the GIR stroke has been taken. (The GIR stroke is determined by subtracting 2 from par (1st stroke on a par 3, 2nd on a par 4, 3rd on a par 5))


Scambling: The percent of time a player misses the green in regulation, but still makes par or better.
Sand Saves: The percent of time a player was able to get 'up and down' once in a greenside sand bunker (regardless of score). Note: 'Up and down' indicates it took the player 2 shots or less to put the ball in the hole from that point.

Putts Per Round:

In [18]:
statsMap = {
  "offical_money" : 109,
  "scoring_average" : 120,
  "driving_distance" : 101,
  "driving_accuracy" : 102,
  "greens_in_regulation":103,
  "proximity_to_the_hole": 331,
  "scrambling": 130,
  "sand_saves": 111,
  "putts_per_round" : 119
}

#loop through the map and create new csv
for title, val in statsMap.items():
    newDf = data_frame_builder(title, val)
    newDf.to_csv(f"../data/raw_data/{title}.csv")